# Lab 7, Weather Data Collection, Cleaning, and Exploration

### Due Aug 2,3, 2022 (in-lab demonstration)

In this lab, you will perform the first three stages of the data science process needed before training a model to predict if there will be `Rain` or `NoRain`

1. Data Collection
1. Data Cleaning
1. Data Exploration


You will used the prepared code below to download hourly data for 1 month from Environment Canada's [historical data site](https://climate.weather.gc.ca/historical_data/search_historic_data_e.html).  The year, month, and day are configuration parameters to Environment Canada's download API (Application Interface)

1. Ensure you read and understand the provided code, comments, and documentation in pre-populated cells

1. Provided needed code to complete the lab and 

1. Answer questions in the cells below


# Notes on Environment Canada Data Quality Indicators

```
'M' Missing
'E' Estimated
'NA' = Not Available
[empty] = Indicates an unobserved value
```

# Notes on Environment Canada API
    timeframe = {
    'hourly':1,
    'daily':2,
    'monthly':3
    }

# Jupyter Settings

Change Jupyter gui for wider cells

Change Pandas to show all columns and rows

In [ ]:
# increase Jupyter cell width

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd

# set pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Environment Canada API Parameter Configuration

In [ ]:
stationID, climateID = "51459", "6158731" # Toronto Pearson

##############################################################################
########### Set Year, Month, Day to capture 1 month of hourly data ###########
year = 2022
month = 6
day = 1
##############################################################################

download_date_str = f"{year}-{month:02d}-{day:02d}"

print("Month data will be downloaded: ", download_date_str)

url_base = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?"
url_hourly = "{url_base}format=csv&stationID={stationID}&Year={year}&Month={month:02d}&Day={day:02d}&timeframe=1".format(url_base=url_base, stationID=stationID, year=year, month=month, day=day)
url_daily = "{url_base}format=csv&stationID={stationID}&Year={year}&Month={month:02d}&Day={day:02d}&timeframe=2".format(url_base=url_base, stationID=stationID, year=year, month=month, day=day)
url_monthly = f"{url_base}format=csv&stationID={stationID}&Year={year}&Month={month:02d}&timeframe=3".format(url_base=url_base, stationID=stationID, year=year, month=month)

# Manual Data Download

Try the links below to download the same data to your PC, open with a text editor (or excel) to examine the fields

In [ ]:
print("Click to download CSV data:")
print(url_hourly)
print(url_daily)
print(url_monthly)

# Automated Data Download

The cells below will download hourly data and save it to a local file on the Pi

Q: what is the name of the local file?

In [ ]:
import requests

In [ ]:
response = requests.get(url_hourly)

In [ ]:
data_download_filepath = f'environment_canada_station_data_{stationID}_{climateID}_{download_date_str}.csv'

with open(data_download_filepath, 'w') as f:
    f.write(response.content.decode("utf-8"))

In [ ]:
!head -n 4 {data_download_filepath}

In [ ]:
df_raw = pd.read_csv(data_download_filepath)

In [ ]:
df_raw.head(2)

# Data Exploration

In [ ]:
# show the column names

your code here

In [ ]:
# show how many rows of data was downloaded

your code here

In [ ]:
# show field data types 

your code here

In [ ]:
# show count of NA's and nan's for each field
df_raw.info()

In [ ]:
# show statistical summary of data fields
df_raw.describe()

# Show weather labels in dataset

In [ ]:
df_raw['Weather'].unique()

# Data Cleaning

In [ ]:
df_cleaned = df_raw.copy()

In [ ]:
# remove special characters from column names
import re
def clean_column_names(column_name):
    col_stage1 = re.sub('\(.*\)','',column_name)
    col_stage2 = re.sub('[. /]','',col_stage1)
    return col_stage2

df_cleaned.columns = [clean_column_names(c) for c in df_cleaned.columns]

In [ ]:
df_cleaned.head(3)

# Data Preparation

- the BMP280 provides temperature and pressure readings.  
- select the temperature, air pressure data fields, and weather labels and save into `df_cleaned` dataframe

In [43]:
# Q: Which additional fields should be selected? 
keep_fields = ['DateTime', ... your code here ... ]
df_cleaned = df_cleaned[keep_fields]

In [ ]:
df_cleaned.head(10)

# Data Cleaning: Clean `Weather` column for later use as labels with weather classification model

Assuming the Weather label does not change quickly from hour-to-hour, replace `NaN`'s with the most recent populated value.

For example, 
- the `Weather` at `2022-06-01 02:00` should be similar to the weather at `2022-06-01 01:00`.  Replace `NaN` with 'Mostly Cloudly'
- the `Weather` at `2022-06-01 03:00` should be similar to the weather at `2022-06-01 02:00`.  Replace `NaN` with 'Mostly Cloudly'
- the `Weather` at `2022-06-01 05:00` should be similar to the weather at `2022-06-01 04:00`.  Replace `NaN` with 'Cloudly'
- the `Weather` at `2022-06-01 06:00` should be similar to the weather at `2022-06-01 05:00`.  Replace `NaN` with 'Cloudly'

Use the `fillna()` function in Pandas to accomplish this


In [ ]:
df_cleaned['Weather_fix'] = your code here # forward fill NA's (previous hour's observation passes into the next hour (ie next row))
df_cleaned['Weather_fix'] = df_cleaned['Weather_fix'].fillna(method='bfill') # backward fill NA's (current hour's observation passes into the previous hour (ie for the first row))

In [ ]:
df_cleaned.head(10)

# Data Preparation: Define PRG550 `Rain/NoRain` labels based on Environment Canada `Weather` labels

In [ ]:
df_cleaned['Weather_fix'].unique()

In [ ]:
# Use the unique Weather labels from the above cell and create a dictionary to map `Weather` label to `Rain` or `NoRain`

prg550_weather_2labels = {

    your code here
    
}

In [ ]:
# create new column with only 2 categories from the above dictionary
df_cleaned['PRG550_2labels'] = your code here
df_cleaned.head(3)

In [ ]:
df_cleaned.loc[df_cleaned['PRG550_2labels']=='Rain'].head(3) # find entries that show Rain

# Data Visualization

- plot visualization of relationship between temperature, air pressure and `Rain`/`NoRain` labels
- use `pairplot()` from the Seaborn library
- use with the parameter `hue="PRG550_2labels"`

See Seaborn documentation to for `pairplot()`:

    https://seaborn.pydata.org/generated/seaborn.pairplot.html

In [ ]:
import seaborn as sns
sns.color_palette("viridis_r", as_cmap=True)
sns.set_palette('tab10')

In [ ]:
_fields = ['Temp','StnPress','PRG550_2labels']
plot_df = df_cleaned[_fields]

sns.pairplot(plot_df, hue='PRG550_2labels')

# Data Analysis

Provide answers to the below questions

1. What is the mean, and standard deviation for temperature and air pressure for the entire dataset?
1. What is the mean, and standard deviation for temperature and air pressure when there is Rain?
1. What is the mean, and standard deviation for temperature and air pressure when there is NoRain?
1. How many data points are there for Rain?
1. How many data points are there for NoRain?


In [ ]:
# Q1 answer

In [ ]:
# Q2 answer

In [ ]:
# Q3 answer

In [ ]:
# Q4 answer

In [ ]:
# Q5 answer